In [55]:
import sys
sys.path.append("..") ## resetting the path to the parent directory
import src.key_reader as kr
api_key = kr.key()

In [39]:
demo_ids = ['4366059781', '4366059798', '4366059814', 
            '4366059822', '4366059865', '4366059892', 
            '4366059918', '4366059925', '4366059934', 
            '4366059955', '4366059956', '4366059965']
demo_id = '4366059865'

In [57]:
def halftime_individual(api_key = api_key):
    import requests
    import pandas as pd
    import json
    import time
    from pathlib import Path

    with open('../data/matchlist_kr.json','r') as file:
        match_ids = json.load(file)

    n = 9231

    api_error_n = 0

    match_ids = match_ids[9231:]
    for match_id in match_ids:

        if n % 90 == 0:
            time.sleep(130)
            pass

        response_t = requests.get(f'https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{match_id}?api_key={api_key}')
        print(n)
        n += 1
        b = response_t.json()

        #keys = ['participantId','totalGold','level','xp','killerId','victimId']
        keys = ['totalGold','level','xp','totalKills','totalDeaths']

        participant_ids = [1,2,3,4,5,6,7,8,9,10]

        team1_halftime_stats = {'match_id': match_id,'totalGold':0,'level':0,
                                'xp':0,'totalKills':0,'totalDeaths':0, 'level_ups':0}

        team2_halftime_stats = {'match_id': match_id,'totalGold':0,'level':0,
                                'xp':0,'totalKills':0,'totalDeaths':0, 'level_ups':0}

        try:
            # Taking first half of the match stats to predict the match outcome.

            for frame_i in range(len(b['frames'])//2):

                ## Grabbing ingame gold, levels, xp    
                for p_i in range(1,len(participant_ids)+1):
                    for ind_key in keys[:-2]:
                        if b['frames'][frame_i]['participantFrames'][str(p_i)]['participantId'] <= 5:
                            team1_halftime_stats[ind_key] += b['frames'][frame_i]['participantFrames'][str(p_i)][ind_key]
                        else:
                            team2_halftime_stats[ind_key] += b['frames'][frame_i]['participantFrames'][str(p_i)][ind_key]

                ## Grabbing ingame kills and deaths
                for event_i in range(len(b['frames'][frame_i]['events'])):
                    for ind_key2 in keys[-2:]:

                        if b['frames'][frame_i]['events'][event_i]['type'] == 'CHAMPION_KILL':
                            if b['frames'][frame_i]['events'][event_i]['killerId'] <= 5:
                                team1_halftime_stats['totalKills'] += 1
                                team2_halftime_stats['totalDeaths'] += 1
                            else:
                                team2_halftime_stats['totalKills'] += 1
                                team1_halftime_stats['totalDeaths'] += 1       
                        if b['frames'][frame_i]['events'][event_i]['type'] =='SKILL_LEVEL_UP':
                            if b['frames'][frame_i]['events'][event_i]['participantId'] <= 5:
                                team1_halftime_stats['level_ups'] += 1
                            else:
                                team2_halftime_stats['level_ups'] += 1
            halftime_stats = []
            halftime_stats.append(team1_halftime_stats)
            halftime_stats.append(team2_halftime_stats)

            # Checking if a csv file exists before appending the data
            my_file = Path("../data/ht_match_ind.csv")

            # Create a csv file if it does not exist
            if not my_file.is_file():
                match_stats_df = pd.DataFrame(halftime_stats)
                match_stats_df.to_csv('../data/ht_match_ind.csv')
                time.sleep(0.125)
                halftime_stats = []
            # Append data onto the existing csv file
            else:
                match_stats_df = pd.DataFrame(halftime_stats)
                match_stats_df.to_csv('../data/ht_match_ind.csv',header=None,mode='a')
                time.sleep(0.125)
                halftime_stats = []
        except:
            print(api_error_n)
            api_error_n += 1
            print(f"error occurred in n'th match: {n}")
            print(response_t.status_code)

            if response_t.status_code == 429 and api_error_n < 3:
                time.sleep(30)
                pass

            elif response_t.status_code == 429 and api_error_n >= 3 and api_error_n < 10:
                time.sleep(30)
                pass

            elif api_error_n >= 10:
                print('breaking due to excessive number of pulls')
                break

            elif response_t.status_code == 504 or response_t.status_code == 503:
                time.sleep(30)
                api_error_n -= 1
                pass
            else:
                time.sleep(30)
                pass        

9231
9232
9233
9234
9235
9236
9237
9238
9239
9240
9241
9242
9243
9244
9245
9246
9247
9248
9249
9250
9251
9252
9253
9254
9255
9256
9257
0
error occurred in n'th match: 9258
504
9258
9259
9260
9261
9262
9263
9264
9265
9266
9267
9268
9269
9270
9271
9272
9273
9274
9275
9276
9277
9278
9279
9280
9281
9282
9283
9284
9285
9286
9287
9288
9289
9290
9291
9292
9293
9294
9295
9296
9297
9298
9299
9300
9301
9302
0
error occurred in n'th match: 9303
504
9303
9304
9305
9306
9307
9308
9309
9310
9311
9312
9313
9314
9315
9316
9317
9318
9319
9320
9321
9322
9323
9324
9325
9326
9327
9328
9329
9330
9331
9332
9333
9334
9335
9336
9337
9338
9339
9340
9341
9342
9343
9344
0
error occurred in n'th match: 9345
503
9345
9346
9347
9348
9349
9350
9351
9352
9353
9354
9355
9356
9357
9358
9359
9360
9361
9362
9363
9364
9365
9366
9367
9368
9369
9370
9371
9372
9373
9374
9375
9376
9377
9378
9379
9380
9381
9382
9383
9384
9385
9386
9387
9388
9389
9390
9391
9392
9393
9394
9395
9396
9397
9398
9399
9400
9401
9402
9403
9404
9405
94